In [2]:
#Clement_Bisaillon
# title,text,subject,date
import pandas as pd

def convert_to_json(file_path_csv, label):
    df = pd.read_csv(file_path_csv, engine="python")
    df.drop(["subject","date"], axis=1, inplace=True)
    df["label"] = label
    return df

db_name = "Clement_Bisaillon"
base_file_path = f"./data/{db_name}/"

pdfake = convert_to_json(base_file_path + "Fake.csv",  0)
pdtrue = convert_to_json(base_file_path + "True.csv", 1)

pdcombined = pd.concat([pdfake, pdtrue], ignore_index=True)
pdcombined.to_json(f"./data/jsons/{db_name}.json", orient="records", indent=2, force_ascii=False)



In [3]:
# Erfan_Moosavi_Monazzah
# title	text	subject	date	label
import pandas as pd

db_name = "Erfan_Moosavi_Monazzah"

base_file_path = f"./data/{db_name}/"

df = pd.read_csv(base_file_path + "test.tsv", sep="\t")

df.drop(["subject","date", "Unnamed: 0"], axis=1, inplace=True)

df.to_json(f"./data/jsons/{db_name}.json", orient="records", indent=2, force_ascii=False)

In [4]:
# Mahdi_Mashayekhi
# title,text,date,source,author,category,label
import pandas as pd

db_name = "Mahdi_Mashayekhi"

base_file_path = f"./data/{db_name}/"
df = pd.read_csv(base_file_path + "fake_news_dataset.csv", engine="python")
df.drop(["date","source","author","category"], axis=1, inplace=True)
df["label"] = df["label"].map({"real": 1, "fake": 0})

df.to_json(f"./data/jsons/{db_name}.json", orient="records", indent=2, force_ascii=False)

In [5]:
# Saurabh_Shahane
# ,title,text,label
import pandas as pd
import csv

csv.field_size_limit(10**7) 

db_name = "Saurabh_Shahane"
base_file_path = f"./data/{db_name}/"

df = pd.read_csv(base_file_path + "WELFake_Dataset.csv", engine="python")
df.drop(["Unnamed: 0"], axis=1, inplace=True)

df.to_json(f"./data/jsons/{db_name}.json", orient="records", indent=2, force_ascii=False)


In [6]:
# Subhajournal
# ,title,text,label
import pandas as pd

db_name = "Subhajournal"
base_file_path = f"./data/{db_name}/"
df = pd.read_csv(base_file_path + "Fake_Real_News_Data.csv", engine="python")

df.drop("Unnamed: 0", axis=1, inplace=True)

df["label"] = df["label"].map({"FAKE": 0, "REAL": 1})

df.to_json(f"./data/jsons/{db_name}.json", orient="records", indent=2, force_ascii=False)

In [7]:
import pandas as pd

input_folder = "./jsons/"
output_file = "./combined_news.json"

dfs_dirs = ["Clement_Bisaillon.json",
        "Erfan_Moosavi_Monazzah.json",
        "Mahdi_Mashayekhi.json",
        "Saurabh_Shahane.json",
        "Subhajournal.json"]

dfs = []

for file in dfs_dirs:
    df = pd.read_json(f"./data/jsons/{file}")
    dfs.append(df)

combined_df = pd.concat(dfs, ignore_index=True)

print("Total rows:", combined_df.shape[0])
combined_df.to_json(output_file, orient="records", indent=2, force_ascii=False)
print("Saved to:", output_file)


Total rows: 151634
Saved to: ./combined_news.json


In [15]:
import nltk
# pip install nltk

# --------------------------
# NLTK setup (make sure you downloaded these once):
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
# --------------------------

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Ibrah\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Ibrah\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Ibrah\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [ ]:
import re

import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer


stop_words = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()

contractions = {
    "i'm": "i am", "im": "i am",
    "i'll": "i will", "ill": "i will",
    "i've": "i have", "ive": "i have",

    "can't": "cannot", "cant": "cannot",
    "aren't": "are not", "arent": "are not",
    "couldn't": "could not", "couldnt": "could not",
    "didn't": "did not", "didnt": "did not",
    "doesn't": "does not", "doesnt": "does not",
    "hadn't": "had not", "hadnt": "had not",
    "hasn't": "has not", "hasnt": "has not",
    "haven't": "have not", "havent": "have not",
    "isn't": "is not", "isnt": "is not",
    "let's": "let us", "lets": "let us",
    "mustn't": "must not", "mustnt": "must not",
    "won't": "will not", "wont": "will not",
    "don't": "do not", "dont": "do not",

    "you're": "you are", "youre": "you are",
    "they're": "they are", "theyre": "they are",
    "she's": "she is", "shes": "she is",
    "he's": "he is", "hes": "he is",
    "we're": "we are", "were": "we are",
    "that's": "that is", "thats": "that is",
    "it's": "it is", "its": "it is",
}

def expand_contractions(text):
    for c, full in contractions.items():
        text = re.sub(r"\b" + re.escape(c) + r"\b", full, text, flags=re.IGNORECASE)
    return text

def normalize_text(text):
    if not isinstance(text, str):
        text = "" if text is None else str(text)

    text = text.lower()
    text = expand_contractions(text)
    # remove punctuation (keeps letters/digits/underscore)
    text = re.sub(r"[^\w\s]", " ", text)

    tokens = text.split()

    # remove stop words + lemmatize
    cleaned_tokens = [
        lemmatizer.lemmatize(tok)
        for tok in tokens
        if tok not in stop_words
    ]

    return " ".join(cleaned_tokens)

def clean_news(in_path, out_path = "cleaned_news.json"):
    df = pd.read_json(in_path)

    print("Columns:", list(df.columns))
    print("Records:", len(df))

    df["title"] = df["title"].apply(normalize_text)
    df["text"] = df["text"].apply(normalize_text)

    df.to_json(out_path, orient="records", indent=2, force_ascii=False)

    print(f"Cleaned JSON written to: {out_path}")

clean_news("combined_news.json")

Columns: ['title', 'text', 'label']
Records: 151634


In [ ]:
# 